# Loading and Imports


In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 89.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
import pandas as pd

In [ ]:
df=pd.read_csv("/content/idmanual.csv")
df

,id_tx,class_id,description,status
0,009-4140,009,Bank note acceptors for separating good bank n...,A
1,009-4136,009,Fingerprint imagers,A
2,009-4133,009,Laboratory swabs [laboratory instruments],A
3,009-4131,009,Ear plugs for divers,A
4,009-4130,009,DVD recorders,A
...,...,...,...,...
58697,004-664,004,Cutting fluids for milling,A
58698,004-665,004,Cutting oils for millworking,A
58699,004-666,004,Cutting oils for milling,A
58700,004-667,004,Palm oil being biodiesel fuel,A


#EDA

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58702 entries, 0 to 58701
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_tx        58702 non-null  object
 1   class_id     58702 non-null  object
 2   description  58702 non-null  object
 3   status       58702 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download('stopwords')
nltk.download('punkt')


def remove_stopwords(text):

    stopWords = set(stopwords.words('english'))
    try:
        words = word_tokenize(text)
    except Exception as e:
        print("### ",e,text,type(text))

    wordsFiltered = []

    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)

    text_filtered=' '.join(wordsFiltered)
    return text_filtered


text_2=[]
for t in df['description']:
    text_filtered=remove_stopwords(t)
    text_2.append(text_filtered)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df['desc']=text_2

In [ ]:
df

,id_tx,class_id,description,status,desc
0,009-4140,009,Bank note acceptors for separating good bank n...,A,Bank note acceptors separating good bank notes...
1,009-4136,009,Fingerprint imagers,A,Fingerprint imagers
2,009-4133,009,Laboratory swabs [laboratory instruments],A,Laboratory swabs [ laboratory instruments ]
3,009-4131,009,Ear plugs for divers,A,Ear plugs divers
4,009-4130,009,DVD recorders,A,DVD recorders
...,...,...,...,...,...
58697,004-664,004,Cutting fluids for milling,A,Cutting fluids milling
58698,004-665,004,Cutting oils for millworking,A,Cutting oils millworking
58699,004-666,004,Cutting oils for milling,A,Cutting oils milling
58700,004-667,004,Palm oil being biodiesel fuel,A,Palm oil biodiesel fuel


In [ ]:
df['class_id'].value_counts()

009    5457
016    2874
037    2485
035    2448
007    2323
028    2107
005    2104
021    2103
001    2094
041    1963
042    1894
010    1850
030    1771
011    1712
029    1694
006    1626
003    1590
020    1562
012    1549
025    1413
044    1206
019    1161
036    1145
008     995
040     886
031     865
045     855
018     792
017     770
039     768
024     612
002     605
026     565
014     564
043     542
038     515
004     513
032     505
022     415
013     404
015     389
033     312
034     301
027     228
023     134
000      32
B         2
A         1
200       1
Name: class_id, dtype: int64

In [ ]:
## removing classes with only single datapoint

df.drop(df.loc[df['class_id'].isin(['A','B','200'])].index,inplace=True)


In [ ]:
df['Class']=df['class_id'].astype(int)

In [ ]:
df

,id_tx,class_id,description,status,desc,Class
0,009-4140,009,Bank note acceptors for separating good bank n...,A,Bank note acceptors separating good bank notes...,9
1,009-4136,009,Fingerprint imagers,A,Fingerprint imagers,9
2,009-4133,009,Laboratory swabs [laboratory instruments],A,Laboratory swabs [ laboratory instruments ],9
3,009-4131,009,Ear plugs for divers,A,Ear plugs divers,9
4,009-4130,009,DVD recorders,A,DVD recorders,9
...,...,...,...,...,...,...
58697,004-664,004,Cutting fluids for milling,A,Cutting fluids milling,4
58698,004-665,004,Cutting oils for millworking,A,Cutting oils millworking,4
58699,004-666,004,Cutting oils for milling,A,Cutting oils milling,4
58700,004-667,004,Palm oil being biodiesel fuel,A,Palm oil biodiesel fuel,4


In [ ]:
df=df[['desc','Class']]
df

,desc,Class
0,Bank note acceptors separating good bank notes...,9
1,Fingerprint imagers,9
2,Laboratory swabs [ laboratory instruments ],9
3,Ear plugs divers,9
4,DVD recorders,9
...,...,...
58697,Cutting fluids milling,4
58698,Cutting oils millworking,4
58699,Cutting oils milling,4
58700,Palm oil biodiesel fuel,4


In [ ]:
df.isnull().sum()

desc     0
Class    0
dtype: int64

#Data Preparation

In [ ]:
from sklearn.model_selection import train_test_split

X, X_test, y, y_test = train_test_split(df.index.values, 
                                                  df.Class.values, 
                                                  test_size=0.01, 
                                                  random_state=21, 
                                                  stratify=df.Class.values)

X_train, X_val, y_train, y_val = train_test_split(X,
                                                  y,
                                                  test_size=0.10, 
                                                  random_state=42, 
                                                  stratify=y)

In [ ]:
X_train.shape

(52299,)

In [ ]:
from sklearn.model_selection import train_test_split


df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'
df.loc[X_test, 'data_type'] = 'test'
df.groupby(['Class', 'data_type']).count()

<ipython-input-28-9b7e90784dd2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['data_type'] = ['not_set']*df.shape[0]


desc
Class data_type      
0     train        29
      val           3
1     test         21
      train      1866
      val         207
...               ...
44    train      1075
      val         119
45    test          9
      train       761
      val          85

[137 rows x 1 columns]

In [ ]:
len(df['Class'].unique())

46

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].desc.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].desc.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].Class.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].Class.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
encoded_data_test = tokenizer.batch_encode_plus(
    df[df.data_type=='test'].desc.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df[df.data_type=='test'].Class.values)

In [ ]:
dataset_test = TensorDataset(input_ids_test, attention_masks_test)

#Model Buliding / Training

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=46,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 8

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 25

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    #label_dict_inverse = {v: k for k, v in label_dict.items()}
    ACC=0
    ACCwom=0
    c=0
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {y_true[0]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')
        acc=len(y_preds[y_preds==label])/len(y_true)
        ACC+=acc
        # if label_dict_inverse[label] !="MEDICATION":
        #     ACCwom+=acc
        c+=1
    print("TOTAL ACCURACY ", ACC/c)

In [ ]:
import random
import numpy as np
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/content/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


cuda


  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/6538 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.3315833909012567
Validation loss: 0.5807408255587705
F1 Score (Weighted): 0.843022028647295


Epoch 2:   0%|          | 0/6538 [00:00<?, ?it/s]

...

Epoch 3:   0%|          | 0/6538 [00:00<?, ?it/s]


Epoch 3
Training loss: 0.3220152844939553
Validation loss: 0.5144616383768553
F1 Score (Weighted): 0.8860762537715907


Epoch 4:   0%|          | 0/6538 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.23307385394024308
Validation loss: 0.5863332077565391
F1 Score (Weighted): 0.8892503641931101


Epoch 5:   0%|          | 0/6538 [00:00<?, ?it/s]

#Evaluation

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


print(device)

cpu


In [ ]:

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=46,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('/content/drive/MyDrive/trademarkia/finetuned_BERT_epoch_3.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

NameError: ignored

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: 0
Accuracy: 0/3

Class: 1
Accuracy: 187/207

Class: 2
Accuracy: 53/60

Class: 3
Accuracy: 133/157

Class: 4
Accuracy: 47/51

Class: 5
Accuracy: 184/208

Class: 6
Accuracy: 151/161

Class: 7
Accuracy: 204/230

Class: 8
Accuracy: 84/99

Class: 9
Accuracy: 474/540

Class: 10
Accuracy: 159/183

Class: 11
Accuracy: 146/170

Class: 12
Accuracy: 140/153

Class: 13
Accuracy: 35/40

Class: 14
Accuracy: 47/56

Class: 15
Accuracy: 35/38

Class: 16
Accuracy: 262/285

Class: 17
Accuracy: 70/76

Class: 18
Accuracy: 67/78

Class: 19
Accuracy: 96/115

Class: 20
Accuracy: 117/155

Class: 21
Accuracy: 174/208

Class: 22
Accuracy: 28/41

Class: 23
Accuracy: 13/13

Class: 24
Accuracy: 51/61

Class: 25
Accuracy: 115/140

Class: 26
Accuracy: 46/56

Class: 27
Accuracy: 19/23

Class: 28
Accuracy: 191/209

Class: 29
Accuracy: 150/168

Class: 30
Accuracy: 142/175

Class: 31
Accuracy: 74/86

Class: 32
Accuracy: 43/50

Class: 33
Accuracy: 26/31

Class: 34
Accuracy: 29/30

Class: 35
Accuracy: 224/242

Class

In [ ]:
true_vals

array([9, 9, 9, ..., 4, 4, 4])